<a href="https://colab.research.google.com/github/elka97/MachineLearningInBusiness/blob/main/homework_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [362]:
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
from sklearn.metrics import precision_recall_curve
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score
import xgboost as xgb

In [363]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [364]:
DATA_PATH = "/gdrive/My Drive/Ml_In_Business/lection6_materials/"
training_dir = f"{DATA_PATH}/catboost_info/" 
DATA_PATH

'/gdrive/My Drive/Ml_In_Business/lection6_materials/'

In [365]:
def evaluate_results(_y_test, _y_predict):   
    _f1 = f1_score(_y_test, _y_predict)   
    _roc = roc_auc_score(_y_test, _y_predict)    
    _rec = recall_score(_y_test, _y_predict, average='binary')  
    _prc = precision_score(_y_test, _y_predict, average='binary')  
    print(f"Classification results: f1: {_f1 * 100.0:.2f}% roc: {_roc * 100.0:.2f}% recall: {_rec * 100.0:.2f}% precision: {_prc * 100.0:.2f}%") 
    return _f1, _roc, _rec, _prc

In [366]:
CLASS_PU = 'class_PU'
CLASS_BINARY = "class_binary"

In [367]:
data = pd.read_csv(f"{DATA_PATH}wine.data")
data.head(3)

,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280/OD315_of_diluted_wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185


In [368]:
print(data.shape)

(178, 14)


In [369]:
data.describe()

,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280/OD315_of_diluted_wines,Proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,1.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,1.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,1.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,2.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,3.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,3.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


Посмотрим на соотношение классов

In [370]:
data["class"].value_counts()

2    71
1    59
3    48
Name: class, dtype: int64

У нас есть 13 признакov и 1 целевая переменная with 3 classes

Join classes 1 and 1 in one target (negative), others - to positive clas, for binary classification

In [371]:
data.loc[data["class"].isin([1,2]), CLASS_BINARY]=0
data.loc[data["class"]==3, CLASS_BINARY]=1
data = data.drop(columns=["class"])

In [372]:
data[CLASS_BINARY].value_counts()

0.0    130
1.0     48
Name: class_binary, dtype: int64

Разбиваем выборку на тренировочную и тестовую части и обучаем модель (в примере - градиентный бустинг)

In [373]:
x_data = data.drop(columns=[CLASS_BINARY])
y_data = data[CLASS_BINARY]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [374]:
y_data.shape, x_data.shape

((178,), (178, 13))

In [375]:
x_train.head(3)

,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280/OD315_of_diluted_wines,Proline
163,12.96,3.45,2.35,18.5,106,1.39,0.70,0.40,0.94,5.28,0.68,1.75,675
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
11,14.12,1.48,2.32,16.8,95,2.20,2.43,0.26,1.57,5.00,1.17,2.82,1280


In [376]:
y_train.head(3)

163    1.0
173    1.0
11     0.0
Name: class_binary, dtype: float64

In [377]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((142, 13), (142,), (36, 13), (36,))

In [378]:
model = model = xgb.XGBClassifier(random_state=22)

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

Проверяем качество

In [379]:
evaluate_results(y_test, y_predict)

Classification results: f1: 95.65% roc: 95.83% recall: 91.67% precision: 100.00%


(0.9565217391304348, 0.9583333333333333, 0.9166666666666666, 1.0)

### random negative sampling 

Представим, что нам неизвестны негативы и часть позитивов

In [380]:
def create_PU_set(df, source_class, PU_class, p=0.25):  
  _df = df.copy()
  source_pos_ind = _df.loc[_df[source_class]==1]

  # get Positive by frac (shuffled)
  source_positive_random_sample = _df.loc[_df[source_class]==1].sample(frac=p, random_state=7)
  # attach target column: all p    
  source_positive_random_sample[PU_class] = 1
  # print(source_positive_random_sample.head(3))
  n_positive_random_sample = source_positive_random_sample.shape[0]

  print(f'Using {n_positive_random_sample} of {source_pos_ind.shape[0]} as positives and unlabeling the rest; {source_positive_random_sample.shape}')

  # add target column: new P and U according to p
  # Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
  _df[PU_class] = -1
  _df.loc[source_positive_random_sample.index, PU_class] = 1
  print(f'{PU_class} new target variable:\n', _df[PU_class].value_counts())

  # get unlabeld by frac (shuffled)
  dest_U_random_sample = _df.loc[_df[PU_class]==-1].sample(n=n_positive_random_sample, random_state=7)
  # print(f'unlabeled fraction: {dest_U_random_sample.shape}')
  # print(dest_U_random_sample.head(3))

  #  concat/shuffle sample
  sample_train = pd.concat([source_positive_random_sample, dest_U_random_sample]).sample(frac=1) 
  print(f"sample_train: {sample_train.shape}")
  # print(sample_train.head(3))
  print(f'{PU_class} in sample_train:\n {sample_train[PU_class].value_counts()}')
 
  _test_df = _df.drop(_df.index[dest_U_random_sample.index])

  return sample_train.drop(columns=[source_class, PU_class]), sample_train[PU_class], _test_df 

In [381]:
fractions_of_p = [0.25, 0.5, 0.75, 1]
results_rows = []
results_idx = []

for pp in fractions_of_p:
  sample_X_train, sample_y_train, sample_test = create_PU_set(data, CLASS_BINARY, CLASS_PU, p=pp)

  model = xgb.XGBClassifier(random_state=22)
  model.fit(sample_X_train, sample_y_train)
  sample_y_predict = model.predict(sample_test.drop(columns=[CLASS_BINARY, CLASS_PU]))
  f1, roc, rec, prc = evaluate_results(sample_test[CLASS_PU], sample_y_predict)
  results_rows.append([f1, roc, rec, prc])
  results_idx.append(pp)
  print("---------------------------------------------------")

Using 12 of 48 as positives and unlabeling the rest; (12, 15)
class_PU new target variable:
 -1    166
 1     12
Name: class_PU, dtype: int64
sample_train: (24, 15)
class_PU in sample_train:
  1    12
-1    12
Name: class_PU, dtype: int64
Classification results: f1: 46.15% roc: 90.91% recall: 100.00% precision: 30.00%
---------------------------------------------------
Using 24 of 48 as positives and unlabeling the rest; (24, 15)
class_PU new target variable:
 -1    154
 1     24
Name: class_PU, dtype: int64
sample_train: (48, 15)
class_PU in sample_train:
  1    24
-1    24
Name: class_PU, dtype: int64
Classification results: f1: 64.86% roc: 90.00% recall: 100.00% precision: 48.00%
---------------------------------------------------
Using 36 of 48 as positives and unlabeling the rest; (36, 15)
class_PU new target variable:
 -1    142
 1     36
Name: class_PU, dtype: int64
sample_train: (72, 15)
class_PU in sample_train:
  1    36
-1    36
Name: class_PU, dtype: int64
Classification re

In [382]:
results = pd.DataFrame(data=results_rows, columns=["f1", "roc", "rec", "prc"], index=results_idx)
results

,f1,roc,rec,prc
0.25,0.461538,0.909091,1.0,0.300000
0.50,0.648649,0.900000,1.0,0.480000
0.75,0.900000,0.962264,1.0,0.818182
1.00,0.969697,0.981707,1.0,0.941176


**чем больше sub-sample Positives, взятых для обучения, тем лучше предсказание (more P and p better prediction)**

при этом recall даже для маленьких fractions (n) при random negative sampling очень хороший, т.. находит все Positives.

При random negative sampling и размере фракции 1 (т.е. все Positive examples которые есть в нашем разпоряжении), F1 получился немного лучше чем одноразовое простое обучение модели: 96.97% vs. 95.65%